## Installing Libraries

In [1]:
import numpy as np
from catboost import CatBoostClassifier, FeaturesData
import pandas as pd
from sklearn.metrics import confusion_matrix
import scipy.io 
import h5py 

## Data Preprocessing 

#### Loading Data

In [2]:
data1 = scipy.io.loadmat(r'C:\Users\mohamedr002\OneDrive\PhD Work\Ph.D Courses\Data Mining\Data Mining Project\Full Features\normalized_data.mat')
data=data1.get('normalizedData')
train_data=data[:, 0:13]
labels=data[:,13]

In [98]:
# # Testing by Removing specific feature for every time 
# specify the feture you want to remove 
train_data=np.delete(train_data, np.s_[12], axis=1) 
print(train_data.shape)

(500000, 12)


#### Processing data 

In [3]:
# folding data and labels into five folds 
def folding_data(data):
    if data.ndim==2:
        f1=data[0:100000,:]
        f2=data[100000:200000,:]
        f3=data[200000:300000,:]
        f4=data[300000:400000,:]
        f5=data[400000:500000,:]
        return (f1,f2,f3,f4,f5)
    elif data.ndim==1:
        f1=data[0:100000]
        f2=data[100000:200000]
        f3=data[200000:300000]
        f4=data[300000:400000]
        f5=data[400000:500000]
        return(f1,f2,f3,f4,f5)
                      

In [6]:
(fold1,fold2,fold3,fold4,fold5)=folding_data(train_data)
(fold1Label,fold2Label,fold3Label,fold4Label,fold5Label)=folding_data(labels)

accuaracy =[]
fold=[fold1.tolist(), fold2.tolist(),fold3.tolist(),fold4.tolist(),fold5.tolist()]
foldlabel=[fold1Label.tolist(),fold2Label.tolist(),fold3Label.tolist(),fold4Label.tolist(),fold5Label.tolist()]
print(type(fold[0]))

<class 'list'>


##  Training and Testing 

In [11]:
for i in range (0,4):
   
    #preprocessing the data for k-fold
    test_data=fold[i]
    temp_fold=fold[:]
    del temp_fold[i]
    train_data = np.concatenate(temp_fold)
    print(train_data.shape)
    
    #preprocessing the labels for k-fold
    test_labels=foldlabel[i]
    temp_label=foldlabel[:]
    del temp_label[i]
    train_labels= np.concatenate(temp_label)
    
    # Initialize CatBoostClassifier
    model = CatBoostClassifier(iterations=5, learning_rate=0.05, depth=6, loss_function='MultiClass')
    # Fit model
    model.fit(train_data, train_labels, plot=True)
    
    # Get predicted classes
    preds_class = model.predict(test_data)
    # Get predicted probabilities for each class
    preds_proba = model.predict_proba(test_data)
    # Get predicted RawFormulaVal
    preds_raw = model.predict(test_data, prediction_type='RawFormulaVal')
    
    
    # Get Confusion matrix
    confusionMat=confusion_matrix(test_labels, preds_class)
    accuaracy.append(np.sum(np.diagonal(confusionMat))/np.sum(confusionMat))